In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from pandasql import sqldf
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
pd.options.mode.chained_assignment = None
import  mysql.connector
pd.options.display.float_format = '{:.5f}'.format

In [142]:
import pyodbc 
import pandas as pd
try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=CASH-MGT-SV01;DATABASE=CashManagement;UID=cashcom_user;PWD=cash@312;')
    cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='latin1')
    cnxn.setencoding('latin1')
    cursor = cnxn.cursor()
    mySql_select_Query = """select  * from  Analytics_cash_MGT_withdrawer_B
                          where REFERENCE_DATE <='2022-06-01'
                             """
    result = cursor.execute(mySql_select_Query)
    queryFetch = cursor.fetchall()
    queryResults = pd.DataFrame.from_records(queryFetch, columns = [i[0] for i in cursor.description])
    print('e don work')
except mysql.connector.Error as error:
    #print(error)
    print("Failed to connect: You go need check am".format(error))

e don work


In [143]:
df=queryResults.copy()

In [144]:
df.head(5)

,REFERENCE_DATE,Branch_code,tran_crncy_code,DEBIT_1_Day_Back,DEBIT_2_Day_Back,DEBIT_3_Day_Back,DEBIT_4_Day_Back,DEBIT_5_Day_Back,DEBIT_6_Day_Back,DEBIT_7_Day_Back,...,Holiday_status_4_Day_Forward,Holiday_status_5_Day_Forward,Holiday_status_6_Day_Forward,Holiday_status_7_Day_Forward,FORCASTT_DATE,PREBUBHOL,POSTBUBHOL,DAY_MONTH,Month,FORCAST_DATE
0,2019-06-01,206,2,0.00000,0.00000,0.00000,700.00000,0.00000,0.00000,0.00000,...,0,1,1,0,2019-06-03,4,4,3,JUNE,2019-06-03
1,2019-06-01,207,2,4000.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0,1,1,0,2019-06-03,4,4,3,JUNE,2019-06-03
2,2019-06-01,208,2,1500.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0,1,1,0,2019-06-03,4,4,3,JUNE,2019-06-03
3,2019-06-01,209,2,50.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0,1,1,0,2019-06-03,4,4,3,JUNE,2019-06-03
4,2019-06-01,207,1,6222700.00000,11705155.00000,0.00000,30422530.00000,13170180.00000,0.00000,0.00000,...,0,1,1,0,2019-06-03,4,4,3,JUNE,2019-06-03


In [145]:
df.tran_crncy_code.unique()

array([Decimal('2'), Decimal('1'), Decimal('4'), Decimal('3')],
      dtype=object)

In [146]:
df = df.apply(pd.to_numeric, errors  = 'ignore', downcast = 'float')

In [147]:
df.Branch_code.unique()

array([206., 207., 208., 209., 210., 211., 202., 201., 203., 204., 205.,
       212., 213.], dtype=float32)

In [148]:
df.tran_crncy_code.unique()

array([2., 1., 4., 3.], dtype=float32)

In [149]:
df.FORCAST_DATE=pd.to_datetime(df.FORCAST_DATE)
Day_name=df.FORCAST_DATE.dt.day_name()
df['Day_name']=Day_name

In [150]:
weekend=['Thursday', 'Friday', 'Monday', 'Wednesday', 'Tuesday']

In [151]:
df=df[df.Day_name.isin(weekend)]

In [152]:
df.Target_Debit.min()

0.0

In [153]:
df=df[['DEBIT_1_Day_Back',
       'DEBIT_2_Day_Back', 'DEBIT_3_Day_Back', 'DEBIT_4_Day_Back',
       'DEBIT_5_Day_Back', 'DEBIT_6_Day_Back', 'DEBIT_7_Day_Back',
       'DEBIT_8_Day_Back', 'DEBIT_9_Day_Back', 'DEBIT_10_Day_Back',
       'DEBIT_11_Day_Back', 'DEBIT_12_Day_Back', 'DEBIT_13_Day_Back',
       'DEBIT_14_Day_Back', 'MAX_DEBIT_14_Day_Back', 'Min_DEBIT_14_Day_Back',
       'Sum_DEBIT_14_Day_Back', 'Avg_DEBIT_14_Day_Back',  'NEXT_Number_1',
       'NEXT_Number_2', 'NEXT_Number_3', 'Previous_Number_1',
       'Previous_Number_2', 'Previous_Number_3','PREBUBHOL',
       'POSTBUBHOL','Target_Debit']]

In [154]:
df.shape

(35550, 27)

In [155]:
df=df[df.Target_Debit>=10000] #150942212.45639953

In [156]:
df.shape

(16351, 27)

In [157]:
y=df.Target_Debit
x=df.drop('Target_Debit',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)
col=list(x.select_dtypes(include=['int64','float32']).columns)

In [158]:

num_cols=Pipeline(steps=[('impute',SimpleImputer(strategy='constant',fill_value=-999)),
                         ('scaler',StandardScaler())])

preprocess=ColumnTransformer(transformers=[('num_cols',num_cols,col)])

pipe=Pipeline(steps=[('preprocess',preprocess),('Linear',LinearRegression())])

In [159]:
pipe.fit(x_train,y_train)
prediction=pipe.predict(x_test)

In [160]:
r_square_t=pipe.score(x_train,y_train)
r_square_te=pipe.score(x_test,y_test)
new=mean_absolute_error(y_test,prediction)
net=mean_squared_error(y_test,prediction)
print('the training score is',r_square_t)
print('the training score on test data',r_square_te)
print('the mean absolute error is ',new)
print('the rmse absolute error is ',sqrt(net))

the training score is 0.7375190144104236
the training score on test data 0.7635457906976851
the mean absolute error is  7371950.0
the rmse absolute error is  13121989.096594159


In [161]:
final_pred=pd.DataFrame({'ACTUAL':y_test,'PREDICTIONS':prediction})
final_pred.head(5)

,ACTUAL,PREDICTIONS
24180,24700.00000,47212.00000
17553,3500000.00000,11140682.00000
38984,1615900.00000,3627116.00000
10644,12000.00000,4095560.00000
26368,18412200.00000,15857772.00000


In [162]:
pickle.dump(pipe,open ('providus_debit','wb'))